# Part 1 - A description of the problem and a discussion of the background.

# Description of the Problem

In Porto Alegre a city of the Southern Part of Brazil, if someone is looking to open a coffee shop near the principals avenues, what are the best spot to open it?


# Discussion of the Background

My client are looking for a good spot to install his coffee shop. It must be profitable, a well looked place with many potencial customers, far from another coffee shop 

We have to assume the customers must passing through the coffee shop to go to work, and they are looking for a good spot to take a break when they got tired from work.

The location of the coffee has a significant impact on the returns.

# Part 2 - A description of the data and how it will be used to solve the problem

# Description of the Data

This project will rely on public data from Foursquare located in [Cafeterias em Porto Alegre](https://pt.foursquare.com/explore?cat=coffee&mode=url&near=Porto%20Alegre%2C%20RS%2C%20Brasil&nearGeoId=72057594041380861)

To explore and target recommended locations across different venues according to the presence of similar facilities, we will access data through FourSquare API 
interface and arrange them as a dataframe for visualization.

# Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

### 1. Collect Inspection Data

In this stage I'm installing and importing the libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.



Setting my account details to develop a valid code

In [2]:
CLIENT_ID = 'DBD1G4TA4ZJX0OLGFLA2VRVSQ11YJAW34TAZTEHGM4D0PKX1' # your Foursquare ID
CLIENT_SECRET = 'MOBFAZIUVBVLOQUOTY1SKZ3PVEPJKKM3ZWR4U3DRVADRIMMB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBD1G4TA4ZJX0OLGFLA2VRVSQ11YJAW34TAZTEHGM4D0PKX1
CLIENT_SECRET:MOBFAZIUVBVLOQUOTY1SKZ3PVEPJKKM3ZWR4U3DRVADRIMMB


#### Let's assume we are in Porto Alegre, let's get the latitude and longitude coordinates.

In [3]:
latitude = -30.0277
longitude = -51.2287


print(latitude, longitude)

-30.0277 -51.2287


##### Now, let's assume that what we want is a coffee shop.
So, let's define a query to search for coffee shop that is within 1000 meters near from Porto Alegre.

In [11]:
search_query = 'Cafeteria'
radius = 1000
print(search_query + ' .... OK!')

Cafeteria .... OK!


In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DBD1G4TA4ZJX0OLGFLA2VRVSQ11YJAW34TAZTEHGM4D0PKX1&client_secret=MOBFAZIUVBVLOQUOTY1SKZ3PVEPJKKM3ZWR4U3DRVADRIMMB&ll=-30.0277,-51.2287&v=20180604&query=Cafeteria&radius=1000&limit=30'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9dff237828ae001bd32c59'},
 'response': {'venues': [{'id': '4d4c542d7929594187064fc7',
    'name': 'Cafeteria Algo a Mais',
    'location': {'address': 'Subsolo Malcon',
     'lat': -30.02974262120043,
     'lng': -51.226115430534534,
     'labeledLatLngs': [{'label': 'display',
       'lat': -30.02974262120043,
       'lng': -51.226115430534534}],
     'distance': 337,
     'cc': 'BR',
     'city': 'Porto Alegre',
     'state': 'RS',
     'country': 'Brasil',
     'formattedAddress': ['Subsolo Malcon', 'Porto Alegre, RS', 'Brasil']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587412838',
    'hasPerk': False},
   {'id': '511558d8e4b0ce6dc580cddd',
    'name': 'Cafeteria Fênix',
    'location': {'address': 'Galer

##### Let's create a dataframe from the data we collect

In [14]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,location.neighborhood,venuePage.id
0,4d4c542d7929594187064fc7,Cafeteria Algo a Mais,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1587412838,False,Subsolo Malcon,-30.029743,-51.226115,"[{'label': 'display', 'lat': -30.0297426212004...",337,BR,Porto Alegre,RS,Brasil,"[Subsolo Malcon, Porto Alegre, RS, Brasil]",NaN,NaN,NaN,NaN
1,511558d8e4b0ce6dc580cddd,Cafeteria Fênix,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1587412838,False,Galeria Lamberts,-30.026341,-51.225702,"[{'label': 'display', 'lat': -30.0263412365417...",326,BR,Porto Alegre,RS,Brasil,"[Galeria Lamberts (Loja 31/33), Porto Alegre, ...",Loja 31/33,90030-000,NaN,NaN
2,4eb3e5d8be7bfc28499cdd75,Cafeteria do Hospital Santa Rita,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1587412838,False,Hospital Santa Rita,-30.030656,-51.220473,"[{'label': 'display', 'lat': -30.0306562320857...",858,BR,Porto Alegre,RS,Brasil,"[Hospital Santa Rita, Porto Alegre, RS, 90035-...",NaN,90035-074,NaN,NaN
3,51cf2865498ee1b893cebded,Cafeteria Mezanino,[],v-1587412838,False,Centro Cultural CEEE Érico Veríssimo,-30.029709,-51.227869,"[{'label': 'display', 'lat': -30.0297089711154...",237,BR,Porto Alegre,RS,Brasil,"[Centro Cultural CEEE Érico Veríssimo, Porto A...",NaN,NaN,NaN,NaN
4,5591409b498ec31df9b70cd4,Cafeteria Factum,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1587412838,False,"Lg. João Amorim de Albuquerque, 60",-30.032100,-51.229465,"[{'label': 'display', 'lat': -30.0321, 'lng': ...",495,BR,Porto Alegre,RS,Brasil,"[Lg. João Amorim de Albuquerque, 60 (Faculdade...",Faculdade Factum,90010-250,Centro Histórico,NaN


In [15]:
dataframe.shape

(30, 19)

In [16]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Cafeteria Algo a Mais,Café,Subsolo Malcon,-30.029743,-51.226115,"[{'label': 'display', 'lat': -30.0297426212004...",337,BR,Porto Alegre,RS,Brasil,"[Subsolo Malcon, Porto Alegre, RS, Brasil]",NaN,NaN,NaN,4d4c542d7929594187064fc7
1,Cafeteria Fênix,Café,Galeria Lamberts,-30.026341,-51.225702,"[{'label': 'display', 'lat': -30.0263412365417...",326,BR,Porto Alegre,RS,Brasil,"[Galeria Lamberts (Loja 31/33), Porto Alegre, ...",Loja 31/33,90030-000,NaN,511558d8e4b0ce6dc580cddd
2,Cafeteria do Hospital Santa Rita,Café,Hospital Santa Rita,-30.030656,-51.220473,"[{'label': 'display', 'lat': -30.0306562320857...",858,BR,Porto Alegre,RS,Brasil,"[Hospital Santa Rita, Porto Alegre, RS, 90035-...",NaN,90035-074,NaN,4eb3e5d8be7bfc28499cdd75
3,Cafeteria Mezanino,None,Centro Cultural CEEE Érico Veríssimo,-30.029709,-51.227869,"[{'label': 'display', 'lat': -30.0297089711154...",237,BR,Porto Alegre,RS,Brasil,"[Centro Cultural CEEE Érico Veríssimo, Porto A...",NaN,NaN,NaN,51cf2865498ee1b893cebded
4,Cafeteria Factum,Coffee Shop,"Lg. João Amorim de Albuquerque, 60",-30.032100,-51.229465,"[{'label': 'display', 'lat': -30.0321, 'lng': ...",495,BR,Porto Alegre,RS,Brasil,"[Lg. João Amorim de Albuquerque, 60 (Faculdade...",Faculdade Factum,90010-250,Centro Histórico,5591409b498ec31df9b70cd4
5,Cafeteria Santa Helena,Coffee Shop,"R. dos Andradas, 1560, Lj. 16",-30.029786,-51.225947,"[{'label': 'display', 'lat': -30.0297855693904...",352,BR,Porto Alegre,RS,Brasil,"[R. dos Andradas, 1560, Lj. 16 (Galeria Malcon...",Galeria Malcon,NaN,NaN,54de4e7b498e49208265aa4c
6,Cafeteria - 5 andar,Café,Rua Caldas Júnior 108,-30.029111,-51.232381,"[{'label': 'display', 'lat': -30.0291110996504...",387,BR,Porto Alegre,RS,Brasil,"[Rua Caldas Júnior 108 (Siqueira Campos), Port...",Siqueira Campos,NaN,NaN,4e5e186ad1640a5975c4b9f2
7,Cafeteria Universitaria,Snack Place,Rua Vigário José Inácio,-30.027983,-51.223101,"[{'label': 'display', 'lat': -30.0279825751354...",540,BR,Porto Alegre,RS,Brasil,[Rua Vigário José Inácio (Avenida Voluntários ...,Avenida Voluntários Da Pátria,NaN,NaN,564f7959498e86dce30d2d1c
8,Cafeteria e Tabacaria TC 35,Café,"R. Senhor dos Passos, 241",-30.029189,-51.222866,"[{'label': 'display', 'lat': -30.0291893149861...",586,BR,Porto Alegre,RS,Brasil,"[R. Senhor dos Passos, 241, Porto Alegre, RS, ...",NaN,NaN,NaN,4f565ee7e4b0861b32b03740
9,Cafeteria Madri,Coffee Shop,NaN,-30.028100,-51.222093,"[{'label': 'display', 'lat': -30.0281, 'lng': ...",638,BR,Porto Alegre,RS,Brasil,"[Porto Alegre, RS, Brasil]",NaN,NaN,NaN,58f1fe410868a239e6dcdfb1


In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=1000,
    color='red',
    fill = False,
).add_to(venues_map)

# add the Cafeterias restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### These points are the coffee shops nearby Porto Alegre Downtown.

##### We are going to use the above data in the following way:
Knowing the locations of existing cafeterias, we can apply Kernel Density Estimation (KDE) to determine the area of influence of the existing cafeterias and discovery where we can install our new cafeteria.